In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import pims
import trackpy as tp
import math
from scipy.misc import imsave
from scipy.misc import imread
import re
################################################
# Read frames which have been data-cleaned (wire_particle_tracking_datacleaning). 
################################################
shot = 120
picL = 18333
picR = 18158
BR = [[5.7149E-08,1.1882E-05,-2.4321E-03],[8.8831E-06,-5.3566E-07,-3.8744E-02],[-1.6469E-03,3.3774E-02,9.9867E-01]]
TR = [[-5.7704E-08,1.1998E-05,-2.1514E-03],[8.9694E-06,5.4087E-07,-3.9328E-02],[-1.7813E-03,3.3894E-02,9.9865E-01]]
TL = [[1.1040E-07,2.2954E-05,-4.1584E-03],[1.7160E-05,-1.0348E-06,6.8654E-02],[-3.4505E-03,-7.3661E-02,9.9490E-01]]
BL = [[-1.0175E-07,2.1155E-05,-4.2910E-03],[1.5815E-05,9.5369E-07,6.2907E-02],[-2.8931E-03,-6.8254E-02,9.9567E-01]]
F_10point = [[  8.34360564e-08,   1.89022714e-05,  -3.73041652e-03],[  1.94501963e-05,   1.19512678e-06,  -5.90813947e-02],[ -3.16737708e-03,   5.30589622e-02,   9.96830069e-01]]
F_big = [[  1.75345969e-07,   1.87353876e-05,  -3.69560210e-03], [  1.92508173e-05,   9.79548859e-07,  -5.72239187e-02],[ -3.15390413e-03,   5.12220927e-02,   9.97034661e-01]]
F = F_big
Matr = F
MatrixL = np.array(Matr)
MatrixR = MatrixL.transpose()
Linex = np.arange(0,384,1)
v0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL_sum.tif'.format(shot,shot,picL))
v0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR_sum.tif'.format(shot,shot,picR))
bk0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL0.tif'.format(shot,shot,picL))
bk0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR0.tif'.format(shot,shot,picR))

In [2]:
tL = pd.read_csv('./Data/Shot{}/trackL3_frame_inv.csv'.format(shot))
tL.head()
#tL['particle'].unique()

,frame,diag_remembered,diag_search_range,diag_subnet,diag_subnet_iterations,diag_subnet_size,ecc,ep,frame.1,mass,particle,raw_mass,signal,size,x,y,range
0,0,NaN,NaN,NaN,NaN,NaN,0.376068,0.0,0,30.790762,0.0,66.0,7.697690,0.866025,163.891026,47.000000,96.922585
1,0,NaN,NaN,NaN,NaN,NaN,0.431373,0.0,0,13.816368,2.0,29.0,3.750157,0.853564,205.900000,74.000000,138.917126
2,0,NaN,NaN,NaN,NaN,NaN,0.419483,0.0,0,141.519079,3.0,345.0,42.238609,0.837576,3.944212,104.976290,217.244897
3,0,NaN,NaN,NaN,NaN,NaN,0.421824,0.0,0,171.520334,5.0,431.0,50.331053,0.840571,304.966628,252.001151,149.594805
4,0,NaN,NaN,NaN,NaN,NaN,0.436170,0.0,0,52.896950,6.0,125.0,15.790134,0.837552,76.048507,254.966418,105.293453


In [3]:
tR = pd.read_csv('./Data/Shot{}/trackR3_frame_inv.csv'.format(shot))
tR.head()

,frame,diag_remembered,diag_search_range,diag_subnet,diag_subnet_iterations,diag_subnet_size,ecc,ep,frame.1,mass,particle,raw_mass,signal,size,x,y,range
0,0,NaN,NaN,NaN,NaN,NaN,0.525253,0.0,0,31.908389,0.0,78.0,8.334281,0.859538,338.059701,64.992537,157.225994
1,0,NaN,NaN,NaN,NaN,NaN,0.391566,0.0,0,54.530009,1.0,130.0,15.001705,0.851405,11.868996,77.877729,100.843407
2,0,NaN,NaN,NaN,NaN,NaN,0.316547,0.0,0,44.290749,2.0,97.0,11.191749,0.864472,247.010753,88.112903,54.155644
3,0,NaN,NaN,NaN,NaN,NaN,0.347458,0.0,0,154.779501,3.0,384.0,42.385771,0.852147,19.087692,110.829231,111.948722
4,0,NaN,NaN,NaN,NaN,NaN,0.404022,0.0,0,181.687322,6.0,466.0,51.434419,0.846704,376.973788,129.917431,157.205940


In [ ]:
## Print out particle track list in the left camera
listL = tL['particle'].unique()
listL = listL.astype(int)
listR = tR['particle'].unique()
listR = listR.astype(int)

In [ ]:
##Try to pair particle track between two cameras.
##Loop particle track in the left camera
#Pair = []
for ilistL in listL:
#for ilistL in range(0,1):
    Pair = []
    ipL0 = int(ilistL)
    TrackL = tL[tL['particle']==ipL0]
    lenL = len(TrackL)
    if ipL0>=620 and lenL>400:
        print(ipL0,lenL)    
        xmaxL = np.max(TrackL['x'])
        xminL = np.min(TrackL['x'])
        ymaxL = np.max(TrackL['y'])
        yminL = np.min(TrackL['y'])
        for ilistR in listR:
            MassDiffSum = 0
            FFSum = 0
            FrameCount = 0
            MassDiffAve = 0
            FFAve = 0
            FFSmallCount = 0
            ipR0 = ilistR
            TrackR = tR[tR['particle']==ipR0] 
            lenR = len(TrackR)
            xmaxR = np.max(TrackR['x'])
            xminR = np.min(TrackR['x'])
            ymaxR = np.max(TrackR['y'])
            yminR = np.min(TrackR['y'])
            for iframeL in TrackL['frame'].unique():
                for iframeR in TrackR['frame'].unique():
                    if(iframeL == iframeR):
                        FrameCount = FrameCount+1
                        TrackL1=TrackL[TrackL['frame']==iframeL]
                        TrackR1=TrackR[TrackR['frame']==iframeR]
                        imassL = TrackL1['mass'].iloc[0]
                        imassR = TrackR1['mass'].iloc[0]
                        MassDiff = (imassL-imassR)*(imassL-imassR)
                        MassDiffSum = MassDiffSum+MassDiff
                        ixL = TrackL1['x'].iloc[0]
                        iyL = TrackL1['y'].iloc[0]  
                        ixR = TrackR1['x'].iloc[0]
                        iyR = TrackR1['y'].iloc[0]                         
                        ipL = [ixL,iyL,1]                       
                        ipR = [ixR,iyR,1]
                        LineL = np.dot(ipR,MatrixL) 
                        LineR = np.dot(ipL,MatrixR) 
                        #LineLy = (-LineL[2]-LineL[0]*Linex)/LineL[1]
                        #LineRy = (-LineR[2]-LineR[0]*Linex)/LineR[1]  
                        dlr = math.fabs(np.dot(ipL,LineL))/math.sqrt(LineL[0]*LineL[0]+LineL[1]*LineL[1])
                        drl = math.fabs(np.dot(ipR,LineR))/math.sqrt(LineR[0]*LineR[0]+LineR[1]*LineR[1])
                        FF = dlr + drl
                        if(FF<5):
                            FFSmallCount = FFSmallCount+1
                        FFSum = FFSum + FF
            if(FrameCount>0):
                MassDiffAve = math.sqrt(MassDiffSum/FrameCount)
                FFAve = FFSum/FrameCount
                templist = [ipL0,ipR0,lenL,lenR,xminL,xmaxL,yminL,ymaxL,xminR,xmaxR,yminR,ymaxR,FrameCount,MassDiffAve,FFAve,FFSmallCount]
                Pair.append(templist) 
                P1 = pd.DataFrame(Pair)
                P1.columns = ['PIDL','PIDR','LenL','LenR','Xmin L','Xmax L','Ymin L','Ymax L','Xmin R','Xmax R','Ymin R','Ymax R','Frame Count','Mass Diff','FF Ave','FF Small Count']
                P1.to_csv("./Data/Shot{}/TrackPair_PID_{}.csv".format(shot,ipL0))

(620, 477)
(621, 416)
(622, 505)
(623, 561)
(634, 521)
(638, 512)
(640, 430)
(641, 484)
